<a href="https://colab.research.google.com/github/patryklatka/Analysis-and-Databases/blob/main/Joining_tables_and_subquries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###In this exercise, I learned how to join tables and learned about the functionality of subqueries.###

In [30]:
dsn_database = "wbauer_adb_2023"   # Specify the name of  Database
dsn_hostname = "pgsql-196447.vipserv.org"  # Specify host name
dsn_port = "5432"                # Specify your port number.
dsn_uid = "wbauer_adb"         # Specify your username.
dsn_pwd = "adb2020"        # Specify your password.

install.packages("RPostgres")
library(DBI)
library(RPostgres)

con <- dbConnect(Postgres(), dbname = dsn_database, host=dsn_hostname, port=dsn_port, user=dsn_uid, password=dsn_pwd)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



**Below I wrote some functions, which I then used to solve the tasks.**

In [31]:
film_in_category<- function(category_id)
{
    # A function that returns the result of a database query of the movie title, language, and category for the given category id.
    # Example of the resulting table:
    # | |title |language |category|
    # |0 |Amadeus Holy |English |Action|.
    #
    # The resulting table is to be sorted by so many movie and language.
    #
    # If the input conditions are not met then the function should return NULL.
    #
    # Parameters:
    # category_id (integer): the value of the category id for which we are executing the query.
    #
    # Returns:
    # DataFrame: the DataFrame containing the results of the query.
    if(is.integer(category_id)){
        film_in_category <- dbGetQuery(con, paste("SELECT film.title title, language.name as language, category.name as category
                        FROM film_category
                        INNER JOIN film ON film_category.film_id = film.film_id
                        INNER JOIN language ON film.language_id = language.language_id
                        INNER JOIN category ON category.category_id = film_category.category_id
                        WHERE film_category.category_id =", category_id,
                        "ORDER BY film.title ASC, language.name ASC "))

        return(film_in_category)
    }
    else{
        return(NULL)
    }
}

In [32]:
number_films_in_category <- function(category_id){
    # Function that returns the result of a database query about the number of movies in the given category by the id of the category.
    # Example of the resulting table:
    # | |category |count|
    # |0 |Action |64 |
    #
    # If the input conditions are not met then the function should return a NULL value.
    #
    # Parameters:
    # category_id (integer): the value of the category id for which we are executing the query.
    #
    # Returns:
    # DataFrame: dataFrame containing the results of the query
    if(is.integer(category_id)){
    number_films_in_category <- dbGetQuery(con, paste("SELECT c.name category, COUNT(f.film_id)
                            FROM category c
                            INNER JOIN film_category fc ON c.category_id = fc.category_id
                            INNER JOIN film f on fc.film_id = f.film_id
                            WHERE c.category_id =", category_id,
                            "GROUP BY c.category_id"))
    return(number_films_in_category)
    }
    else{
        return(NULL)
    }
}

In [33]:
number_film_by_length <- function(min_length, max_length){
    # Function that returns the result of the database query for the number of videos for each length between the min_length and max_length values.
    # Example of the resulting table:
    # | |length |count|
    # |0 |46 |64 |
    #
    # If the input conditions are not met then the function should return a NULL value.
    #
    # Parameters:
    # min_length (int,double): value of the minimum length of the movie
    # max_length (int,double): value of the maximum length of the movie
    #
    # Returns:
    # pd.DataFrame: dataFrame containing the results of the query
    if((is.integer(min_length) | is.double(min_length)) & (is.integer(max_length) | is.double(max_length)) & min_length < max_length){
    number_film_by_length <- dbGetQuery(con, paste("SELECT film.length as length, COUNT(film.title) as count
                            FROM category
                            INNER JOIN film_category ON category.category_id = film_category.category_id
                            INNER JOIN film on film_category.film_id = film.film_id
                            WHERE film.length BETWEEN", min_length ,"and",  max_length,
                            "GROUP BY film.length"))
    return(number_film_by_length)
    }
    else{
        return(NULL)
    }
}

In [34]:
client_from_city<- function(city){
    # Function that returns the result of querying the database for a list of customers from the specified city by the city value.
    # Example of the resulting table:
    # | |city |first_name |last_name
    # |0 |Athenai |Linda |Williams
    #
    # The resulting table is to be sorted by customer's last name and first name.
    #
    # If the input conditions are not met then the function should return NULL.
    #
    # Parameters:
    # city (character): the name of the city for which we are to make a list of customers.
    #
    # Returns:
    # dataFrame: dataFrame containing the results of the query
    if(is.character(city)){
    client_from_city <- dbGetQuery(con, paste("SELECT c.city, cus.first_name, cus.last_name
                               FROM customer cus
                               INNER JOIN address ad ON cus.address_id = ad.address_id
                               INNER JOIN city c ON ad.city_id = c.city_id
                               WHERE c.city = '", city, "'", sep = ""))
    return(client_from_city)
    }
    else{
        return(NULL)
    }
}

In [35]:
avg_amount_by_length<-function(length){
    # A function that returns the result of a database query of the average value of movie rentals for a given length of length.
    # Example of the resulting table:
    # | |length |avg
    # |0 |48 |4.295389
    #
    #
    # If the input conditions are not met then the function should return a NULL value.
    #
    # Parameters:
    # length (integer,double): the length of the movie for which we should borrow the average value of borrowed movies.
    #
    # Returns:
    # DataFrame: dataFrame containing the results of the query
    if(is.integer(length) | is.double(length)){
    avg_amount_by_length <- dbGetQuery(con, paste("SELECT f.length, AVG(p.amount)
                                   FROM film f
                                   INNER JOIN inventory i ON f.film_id = i.film_id
                                   INNER JOIN rental r ON i.inventory_id = r.inventory_id
                                   INNER JOIN payment p  ON r.rental_id = p.rental_id
                                   WHERE f.length =", length,
                                   "GROUP BY f.length"))
    return(avg_amount_by_length)
    }
    else{
        return(NULL)
    }
}

In [36]:
client_by_sum_length<-function(sum_min){
    # A function that returns the result of a database query about the total time of rented movies by customers above a given value .
    # Example of the resulting table:
    # | |first_name |last_name |sum
    # |0 |Brian |Wyman |1265
    #
    # The resulting table should be sorted by sum, first_name and customer name.
    # If the input conditions are not met then the function should return NULL.
    #
    # Parameters:
    # sum_min (integer,double): the minimum value of the sum of the length of the rented movies that the customer must meet.
    #
    # Returns:
    # DataFrame: dataFrame containing the results of the query.
    if(is.numeric(sum_min)){
    client_by_sum_length <-dbGetQuery(con, paste("SELECT  cus.first_name, cus.last_name, SUM(f.length)
                               FROM customer cus
                               INNER JOIN rental r ON cus.customer_id = r.customer_id
                               INNER JOIN inventory i ON r.inventory_id = i.inventory_id
                               INNER JOIN film f ON i.film_id = f.film_id
                               GROUP BY cus.first_name, cus.last_name
                               HAVING SUM(f.length) >=", sum_min,
                                "ORDER BY sum(f.length), cus.last_name, cus.first_name "
                               ))

    return(client_by_sum_length)
    }
    else{
        return(NULL)
    }
}

In [37]:
category_statistic_length<-function(name){
    # Function that returns the result of a query to the database about the statistics of the length of movies in the category with the given name.
    # Example of the resulting table:
    # | |category |avg |sum |min |max
    # |0 |Action |111.60 |7143 |47 |185
    #
    # If the input conditions are not met then the function should return NULL.
    #
    # Parameters:
    # name (character): The name of the category for which the statistic is to be output.
    #
    # Returns:
    # DataFrame: DataFrame containing the results of the query.
    if(is.character(name)){
    category_statistic_length <- dbGetQuery(con, paste("SELECT c.name as category, AVG(f.length), SUM(f.length), MIN(f.length), MAX(f.length)
                                        FROM film f
                                        INNER JOIN film_category fc ON f.film_id = fc.film_id
                                        INNER JOIN category c ON fc.category_id = c.category_id
                                        GROUP BY c.name
                                        HAVING c.name = '", name, "'", sep=""))
    return(category_statistic_length)
    }
    else{
        return(NULL)
    }
}

**Find a list of all videos of the same length.**

In [38]:
solution <- dbGetQuery(con, "SELECT title, length
                    FROM film
                    WHERE length = 102")
print(solution)

                title length
1        Boiled Dares    102
2       Falcon Volume    102
3      Giant Troopers    102
4  Heavyweights Beast    102
5     Jeopardy Encino    102
6            Opus Ice    102
7          Panic Club    102
8        Robbers Joon    102
9       Rock Instinct    102
10      Scarface Bang    102
11   Treasure Command    102


**Find all customers living in the same city.**

In [39]:
solution <- dbGetQuery(con, "SELECT city, first_name, last_name
              FROM customer
              INNER JOIN address ON customer.address_id = address.address_id
              INNER JOIN city ON address.city_id = city.city_id
              WHERE city = 'Abu Dhabi'
              ORDER BY first_name ASC")
print(solution)

       city first_name last_name
1 Abu Dhabi        Tom    Milner


**Calculate the average cost of renting all movies.**

In [40]:
solution <- dbGetQuery(con, "SELECT AVG(payment.amount)
                   FROM payment")
print(solution)

       avg
1 4.200606


**Calculate and display the number of movies in all categories.**

In [41]:
solution <- dbGetQuery(con, "SELECT cat.name, COUNT(f.film_id)
                   FROM category cat
                   INNER JOIN  film_category fc ON cat.category_id = fc.category_id
                   INNER JOIN film f ON fc.film_id = f.film_id
                   GROUP BY cat.name
                   ")
print(solution)

          name count
1       Family    69
2        Games    61
3    Animation    66
4     Classics    57
5  Documentary    68
6          New    63
7       Sports    74
8     Children    60
9        Music    51
10      Travel    57
11     Foreign    73
12       Drama    62
13      Horror    56
14      Action    64
15      Sci-Fi    61
16      Comedy    58


**Display the number of all customers grouped by country.**

In [42]:
solution <- dbGetQuery(con, "SELECT ct.country, COUNT(cus.customer_id)
                   FROM customer cus
                   INNER JOIN address ad ON cus.address_id = ad.address_id
                   INNER JOIN city c ON ad.city_id = c.city_id
                   INNER JOIN country ct ON c.country_id = ct.country_id
                   GROUP BY ct.country_id
                   ORDER BY COUNT(cus.customer_id) DESC")
print(solution)

                                  country count
1                                   India    60
2                                   China    53
3                           United States    36
4                                   Japan    31
5                                  Mexico    30
6                      Russian Federation    28
7                                  Brazil    28
8                             Philippines    20
9                                  Turkey    15
10                              Indonesia    14
11                              Argentina    13
12                                Nigeria    13
13                           South Africa    11
14                                 Taiwan    10
15                         United Kingdom     9
16                                   Iran     8
17                                 Poland     8
18                                Germany     7
19                              Venezuela     7
20                                  Ital

**Display information about a store that has more than 100 customers and less than 300 customers.**

In [43]:
solution <- dbGetQuery(con, "SELECT store_id, count(customer_id)
                FROM customer
                GROUP BY store_id
                HAVING count(customer_id) BETWEEN 100 AND 300")
print(solution)

  store_id count
1        2   273


**Select all customers who have watched movies for more than 200 hours**

In [44]:
solution <- dbGetQuery(con, "SELECT cus.first_name, cus.last_name, SUM(f.length)
                               FROM customer cus
                               INNER JOIN rental r ON cus.customer_id = r.customer_id
                               INNER JOIN inventory i ON r.inventory_id = i.inventory_id
                               INNER JOIN film f ON i.film_id = f.film_id
                               GROUP BY cus.first_name, cus.last_name
                               HAVING SUM(f.length) >= 12000")
print(solution)

[1] first_name last_name  sum       
<0 rows> (or 0-length row.names)


**Calculate the average value of a movie rental.**

In [45]:
solution<- dbGetQuery(con, "SELECT  f.title, AVG(pay.amount)
                                   FROM payment pay
                                   INNER JOIN rental r ON pay.rental_id = r.rental_id
                                   INNER JOIN inventory i ON i.inventory_id = r.inventory_id
                                   INNER JOIN film f ON f.film_id = i.film_id
                                   GROUP BY f.title")
print(solution)

                          title      avg
1            Graceland Dynamite 6.323333
2                      Opus Ice 6.090000
3              Braveheart Human 3.390000
4                Wonderful Drop 5.865000
5               Rush Goodfellas 2.918571
6                  Purple Movie 4.406667
7                 Minority Kiss 2.324167
8                    Luke Mummy 4.132857
9              Fantasy Troopers 1.704286
10               Grinch Massage 5.490000
11             Gaslight Crusade 3.927500
12         Microcosmos Paradise 3.656667
13                  Saturn Name 5.451538
14            Memento Zoolander 6.836154
15                 Sunset Racer 1.704286
16            Zoolander Fiction 4.456667
17                  Carol Texas 4.695882
18                 Arabia Dogma 1.823333
19                 Minds Truman 7.621579
20                Stepmom Dream 5.253158
21              Baked Cleopatra 5.123333
22                  Closer Bang 6.365000
23                Enough Raging 3.240000
24             G

**Calculate the average value of movie length in all categories.**

In [46]:
solution <- dbGetQuery(con, "SELECT cat.name, AVG(f.length)
                   FROM category cat
                   INNER JOIN  film_category fc ON cat.category_id = fc.category_id
                   INNER JOIN film f ON fc.film_id = f.film_id
                   GROUP BY cat.name")
print(solution)

          name      avg
1       Family 114.7826
2        Games 127.8361
3    Animation 111.0152
4     Classics 111.6667
5  Documentary 108.7500
6          New 111.1270
7       Sports 128.2027
8     Children 109.8000
9        Music 113.6471
10      Travel 113.3158
11     Foreign 121.6986
12       Drama 120.8387
13      Horror 112.4821
14      Action 111.6094
15      Sci-Fi 108.1967
16      Comedy 115.8276


**Find the longest movie titles in all categories.**

In [47]:
solution<- dbGetQuery(con, "SELECT cat.name, f.title, f.length
                   FROM category cat
                   INNER JOIN  film_category fc ON cat.category_id = fc.category_id
                   INNER JOIN film f ON fc.film_id = f.film_id
                    WHERE f.length IN (SELECT MAX(f.length)
                                       FROM category cat
                                       INNER JOIN  film_category fc ON cat.category_id = fc.category_id
                                       INNER JOIN film f ON fc.film_id = f.film_id
                                        GROUP BY cat.name)
                    ORDER BY cat.name")
print(solution)

          name              title length
1       Action     Darn Forrester    185
2       Action       Worst Banger    185
3    Animation     Sons Interview    184
4    Animation     Lawless Vision    181
5    Animation        Gangs Pride    185
6    Animation     Theory Mermaid    184
7    Animation       Pond Seattle    185
8     Children        Fury Murder    178
9     Children     Wrong Behavior    178
10    Classics  Conspiracy Spirit    184
11      Comedy     Control Anthem    185
12 Documentary          Wife Turn    183
13 Documentary     Young Language    183
14       Drama   Wardrobe Phantom    178
15       Drama      Jacket Frisco    181
16      Family     King Evolution    184
17     Foreign     Innocent Usual    178
18     Foreign      Catch Amistad    183
19     Foreign   Crystal Breaking    184
20     Foreign    Hotel Happiness    181
21     Foreign     Intrigue Worst    181
22     Foreign    Madness Attacks    178
23     Foreign   Rocketeer Mother    178
24     Foreign  

**Find the longest movie in all categories. Compare the result with point 10.**

In [48]:
solution <- dbGetQuery(con, "SELECT title, length
                   from film
                   WHERE length = (SELECT MAX(length)
                                    FROM film)")
print(solution)

                title length
1       Chicago North    185
2      Control Anthem    185
3      Darn Forrester    185
4         Gangs Pride    185
5           Home Pity    185
6       Muscle Bright    185
7        Pond Seattle    185
8  Soldiers Evolution    185
9   Sweet Brotherhood    185
10       Worst Banger    185
